In [1]:
from azureml.core import Workspace

ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep='\n')

ram
ram
centralindia
82cfc0a9-b871-4f48-bb9a-0f9e00faacf9


In [3]:
import joblib
import sklearn

from sklearn.datasets import load_diabetes
from sklearn.linear_model import BayesianRidge, Ridge

In [4]:
x, y = load_diabetes(return_X_y=True)

In [5]:
first_model = Ridge().fit(x, y)

In [6]:
joblib.dump(first_model, "model_ridge.pkl")

['model_ridge.pkl']

In [7]:
from azureml.core.model import Model

In [8]:
model = Model.register(model_path="model_ridge.pkl",
                            model_name="model_ridge",
                            workspace=ws)

Registering model model_ridge


In [9]:
%%writefile score2.py
import joblib
import json
import numpy as np

from azureml.core.model import Model

def init():
    global model_3
    model_3_path = Model.get_model_path(model_name='model_ridge')
    model_3 = joblib.load(model_3_path)

def run(raw_data):
    try:
        data = json.loads(raw_data)['data']
        data = np.array(data)
        result_1 = model_3.predict(data)
        
        return {"prediction1": result_1.tolist()}
    except Exception as e:
        result = str(e)
        return result

Overwriting score2.py


In [10]:
from azureml.core import Environment

env = Environment("deploytocloudenv")
env.python.conda_dependencies.add_pip_package("joblib")
env.python.conda_dependencies.add_pip_package("numpy==1.23")
env.python.conda_dependencies.add_pip_package("scikit-learn=={}".format(sklearn.__version__))

In [11]:
from azureml.core.model import InferenceConfig

inference_config = InferenceConfig(entry_script="score2.py", environment=env)

In [13]:
from azureml.core.webservice import AciWebservice

aci_service_name = "aciservice-modelridge"

deployment_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)

service = Model.deploy(ws, aci_service_name, [model], inference_config, deployment_config, overwrite=True)
service.wait_for_deployment(True)

print(service.state)

/tmp/ipykernel_7498/2843957594.py:7: FutureWarning: azureml.core.model:
To leverage new model deployment capabilities, AzureML recommends using CLI/SDK v2 to deploy models as online endpoint, 
please refer to respective documentations 
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoints /
https://docs.microsoft.com/azure/machine-learning/how-to-attach-kubernetes-anywhere 
For more information on migration, see https://aka.ms/acimoemigration. 
To disable CLI/SDK v1 deprecation warning set AZUREML_LOG_DEPRECATION_WARNING_ENABLED to 'False'
  service = Model.deploy(ws, aci_service_name, [model], inference_config, deployment_config, overwrite=True)


Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2023-02-03 09:37:21+00:00 Creating Container Registry if not exists.
2023-02-03 09:37:21+00:00 Registering the environment.
2023-02-03 09:37:23+00:00 Use the existing image.
2023-02-03 09:37:23+00:00 Generating deployment configuration.
2023-02-03 09:37:24+00:00 Submitting deployment to compute.
2023-02-03 09:37:33+00:00 Checking the status of deployment aciservice-modelridge..
2023-02-03 09:38:49+00:00 Checking the status of inference endpoint aciservice-modelridge.
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy


In [14]:
import json

In [42]:
test_sample = json.dumps({'data': x[0:2].tolist()})
predictions = service.run(test_sample)
predictions

{'prediction1': [182.67357342863968, 90.99902728640282]}